In [1]:
import pandas as pd
import numpy as np
import cv2
import pytesseract
import PIL

In [2]:
img = cv2.imread('images/1.jpg')
cv2.imshow('',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
text = pytesseract.image_to_string(img)

In [4]:
print(text)

“mm INVOICE

# 10001
Date: May 8, 2022

Bill To:

Saifullah Balance Due: 9,900.00
Item Quantity atelc Amount
digital marketing services 1 5,000.00 5,000.00
content writing 1 %4,000.00 %4,000.00

Subtotal: %9,000.00
Tax (10%): =900.00

Total: 9,900.00



In [5]:
data = pytesseract.image_to_data(img)

In [6]:
#data.split('\n')
datalist = list(map(lambda x:x.split('\t'),data.split('\n')))
df = pd.DataFrame(datalist[1:],columns=datalist[0])

In [7]:
df.head(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,2550,3300,-1,
1,2,1,1,0,0,0,207,112,2182,94,-1,
2,3,1,1,1,0,0,207,112,2182,94,-1,
3,4,1,1,1,1,0,207,112,2182,94,-1,
4,5,1,1,1,1,1,207,112,114,39,25,“mm
5,5,1,1,1,1,2,1921,115,468,91,95,INVOICE
6,2,1,2,0,0,0,188,249,2182,1176,-1,
7,3,1,2,1,0,0,1820,249,550,261,-1,
8,4,1,2,1,1,0,2198,249,172,38,-1,
9,5,1,2,1,1,1,2198,251,26,36,93,#


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   level      58 non-null     object
 1   page_num   57 non-null     object
 2   block_num  57 non-null     object
 3   par_num    57 non-null     object
 4   line_num   57 non-null     object
 5   word_num   57 non-null     object
 6   left       57 non-null     object
 7   top        57 non-null     object
 8   width      57 non-null     object
 9   height     57 non-null     object
 10  conf       57 non-null     object
 11  text       57 non-null     object
dtypes: object(12)
memory usage: 5.6+ KB


In [9]:
df.dropna(inplace=True)
df_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height','conf']
df[df_int] = df[df_int].astype(int) 

In [10]:
df.dtypes

level         int64
page_num      int64
block_num     int64
par_num       int64
line_num      int64
word_num      int64
left          int64
top           int64
width         int64
height        int64
conf          int64
text         object
dtype: object

In [11]:
image = img.copy()
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    if level=='page':
        if l==1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
    elif level=='block':
        if l==2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    elif level=='para':
        if l==3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    elif level=='line':
        if l==4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
    elif level=='word':
        if l==5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
cv2.imshow("",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
            